# Black Litterman Allocation Model - Goldman Sachs 1999 Paper

## Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy import linalg
import warnings
warnings.filterwarnings('ignore')

# Set display options
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.6f' % x)

## Load Volatility and Correlation Data

In [2]:
# Note: Update the file path to match your local directory
file_path = './BL_GSachs.xlsx'

# Load market data
Mkt_data = pd.read_excel(file_path, 
                         sheet_name='BL Goldman Sachs paper', 
                         usecols='A:D', 
                         skiprows=4, 
                         nrows=7)

print("Market Data:")
print(Mkt_data)

Market Data:
     Country       vol         w       ER
0  Australia 16.000000  1.600000 3.900000
1     Canada 20.300000  2.200000 6.900000
2     France 24.800000  5.200000 8.400000
3    Germany 27.100000  5.500000 9.000000
4      Japan 21.000000 11.600000 4.300000
5         UK 20.000000 12.400000 6.800000
6        USA 18.700000 61.500000 7.600000


In [3]:
# Load correlation matrix
Mkt_cor = pd.read_excel(file_path, 
                        sheet_name='BL Goldman Sachs paper', 
                        usecols='B:H', 
                        skiprows=15, 
                        nrows=7,
                        header=None).values

# Set row and column names
countries = Mkt_data['Country'].values
Mkt_cor_df = pd.DataFrame(Mkt_cor, index=countries, columns=countries)

print("\nCorrelation Matrix:")
print(Mkt_cor_df)



Correlation Matrix:
           Australia   Canada   France  Germany    Japan       UK      USA
Australia   1.000000 0.488000 0.478000 0.515000 0.439000 0.512000 0.491000
Canada      0.488000 1.000000 0.664000 0.655000 0.310000 0.608000 0.779000
France      0.478000 0.664000 1.000000 0.861000 0.355000 0.783000 0.668000
Germany     0.515000 0.655000 0.861000 1.000000 0.354000 0.777000 0.653000
Japan       0.439000 0.310000 0.355000 0.354000 1.000000 0.405000 0.306000
UK          0.512000 0.608000 0.783000 0.777000 0.405000 1.000000 0.652000
USA         0.491000 0.779000 0.668000 0.653000 0.306000 0.652000 1.000000


## Extract Market Parameters

In [4]:
# Asset volatility
vol = Mkt_data['vol'].values / 100

# Portfolio weights
w0 = Mkt_data['w'].values / 100

# CAPM expected return
mu = Mkt_data['ER'].values / 100

# Correlation matrix
C = Mkt_cor

print("Asset Volatility:")
print(pd.Series(vol, index=countries))
print("\nPortfolio Weights:")
print(pd.Series(w0, index=countries))
print("\nExpected Returns:")
print(pd.Series(mu, index=countries))

Asset Volatility:
Australia   0.160000
Canada      0.203000
France      0.248000
Germany     0.271000
Japan       0.210000
UK          0.200000
USA         0.187000
dtype: float64

Portfolio Weights:
Australia   0.016000
Canada      0.022000
France      0.052000
Germany     0.055000
Japan       0.116000
UK          0.124000
USA         0.615000
dtype: float64

Expected Returns:
Australia   0.039000
Canada      0.069000
France      0.084000
Germany     0.090000
Japan       0.043000
UK          0.068000
USA         0.076000
dtype: float64


## Calculate Covariance Matrix

In [5]:
# Create a diagonal matrix of standard deviations
diag_sd = np.diag(vol)

# Calculate the variance-covariance matrix
Sigma = diag_sd @ C @ diag_sd

print("Covariance Matrix:")
print(pd.DataFrame(Sigma, index=countries, columns=countries))



Covariance Matrix:
           Australia   Canada   France  Germany    Japan       UK      USA
Australia   0.025600 0.015850 0.018967 0.022330 0.014750 0.016384 0.014691
Canada      0.015850 0.041209 0.033428 0.036034 0.013215 0.024685 0.029572
France      0.018967 0.033428 0.061504 0.057866 0.018488 0.038837 0.030979
Germany     0.022330 0.036034 0.057866 0.073441 0.020146 0.042113 0.033092
Japan       0.014750 0.013215 0.018488 0.020146 0.044100 0.017010 0.012017
UK          0.016384 0.024685 0.038837 0.042113 0.017010 0.040000 0.024385
USA         0.014691 0.029572 0.030979 0.033092 0.012017 0.024385 0.034969


## Portfolio Statistics

In [6]:
# Portfolio standard deviation
sd_P = np.sqrt(w0.T @ Sigma @ w0)
print(f"Portfolio Standard Deviation: {sd_P:.6f}")

# Portfolio return
Rp = np.sum(mu * w0)
print(f"Portfolio Return: {Rp:.6f}")

Portfolio Standard Deviation: 0.168926
Portfolio Return: 0.071620


## Risk Aversion Coefficient

In [7]:
# Target Sharpe Ratio
SR = 0.25

# Risk-aversion coefficient
A = SR / sd_P
print(f"Risk Aversion Coefficient (A): {A:.6f}")

# Risk-free rate
rf = 0.03
print(f"Risk-free Rate: {rf:.2f}")

Risk Aversion Coefficient (A): 1.479938
Risk-free Rate: 0.03


## Implied Returns

In [8]:
# Implied return coherent with initial portfolio allocation w0
mu_imp = rf + (Sigma @ w0) * A

print("Implied Returns:")
print(pd.Series(mu_imp, index=countries))

# Alternative calculation (should give same result)
mu_imp_alt = rf + SR * (Sigma @ w0) / sd_P
print("\nImplied Returns (Alternative):")
print(pd.Series(mu_imp_alt, index=countries))

Implied Returns:
Australia   0.053309
Canada      0.070936
France      0.079478
Germany     0.083439
Japan       0.055472
UK          0.070063
USA         0.074754
dtype: float64

Implied Returns (Alternative):
Australia   0.053309
Canada      0.070936
France      0.079478
Germany     0.083439
Japan       0.055472
UK          0.070063
USA         0.074754
dtype: float64


In [9]:
# Risk premium
risk_premium = mu_imp - rf
print("Risk Premium:")
print(pd.Series(risk_premium, index=countries))

# Alternative calculation
risk_premium_alt = SR * (Sigma @ w0) / sd_P
print("\nRisk Premium (Alternative):")
print(pd.Series(risk_premium_alt, index=countries))

Risk Premium:
Australia   0.023309
Canada      0.040936
France      0.049478
Germany     0.053439
Japan       0.025472
UK          0.040063
USA         0.044754
dtype: float64

Risk Premium (Alternative):
Australia   0.023309
Canada      0.040936
France      0.049478
Germany     0.053439
Japan       0.025472
UK          0.040063
USA         0.044754
dtype: float64


## Specifying Portfolio Manager Views

In [10]:
# Link Matrix (P)
# Each row represents a view
# View 1: Germany (index 2) will have 7.5% return
# View 2: UK (index 3) will have 8.5% return
# View 3: France (index 5) will have 5.5% return

P = np.array([
    [0, 0, 1, 0, 0, 0, 0],  # View on Germany
    [0, 0, 0, 1, 0, 0, 0],  # View on UK
    [0, 0, 0, 0, 0, 1, 0]   # View on France
])

view_names = ['L1', 'L2', 'L3']
P_df = pd.DataFrame(P, index=view_names, columns=countries)
print("Link Matrix (P):")
print(P_df)

Link Matrix (P):
    Australia  Canada  France  Germany  Japan  UK  USA
L1          0       0       1        0      0   0    0
L2          0       0       0        1      0   0    0
L3          0       0       0        0      0   1    0


In [11]:
# Views on expected returns
Q = np.array([0.075, 0.085, 0.055]) 

print("Views (Q):")
print(pd.Series(Q, index=view_names))

Views (Q):
L1   0.075000
L2   0.085000
L3   0.055000
dtype: float64


In [12]:
(mu[1] + mu[6]) / 2

np.float64(0.07250000000000001)

In [13]:
# Uncertainty in views (Omega matrix)
omega = np.array([
    [0.0**2, 0, 0],
    [0, 0.0**2, 0],
    [0, 0, 0.0**2]
])

omega_df = pd.DataFrame(omega, index=view_names, columns=view_names)
print("Omega Matrix (View Uncertainty):")
print(omega_df)

Omega Matrix (View Uncertainty):
         L1       L2       L3
L1 0.000000 0.000000 0.000000
L2 0.000000 0.000000 0.000000
L3 0.000000 0.000000 0.000000


## Black-Litterman Expected Returns

In [14]:
# Tau parameter
tau = 1
rho = tau * Sigma

print(f"Tau: {tau}")

Tau: 1


In [15]:
# Conditional expected returns (mu_bar)
# mu_bar = mu_imp + rho * P' * inv(P * rho * P' + omega) * (Q - P * mu_imp)

term1 = P @ rho @ P.T + omega
term2 = np.linalg.inv(term1)
term3 = Q - P @ mu_imp

mu_bar = mu_imp + rho @ P.T @ term2 @ term3

print("Black-Litterman Expected Returns:")
print(pd.Series(mu_bar, index=countries))

print("\nDifference from Implied Returns:")
print(pd.Series(mu_bar - mu_imp, index=countries))

Black-Litterman Expected Returns:
Australia   0.050181
Canada      0.067649
France      0.075000
Germany     0.085000
Japan       0.050405
UK          0.055000
USA         0.070008
dtype: float64

Difference from Implied Returns:
Australia   -0.003129
Canada      -0.003287
France      -0.004478
Germany      0.001561
Japan       -0.005067
UK          -0.015063
USA         -0.004745
dtype: float64


## Portfolio Optimization (Markowitz)

In [21]:
def Markowitz(mu, Sigma, lmd=0.5, A=1.0, rf=0.03):
    """
    Solve the Markowitz portfolio optimization problem using CVXPY.
    
    Parameters:
    -----------
    mu : array
        Expected returns
    Sigma : array
        Covariance matrix
    lmd : float
        Lambda parameter (default: 0.5)
    A : float
        Risk aversion coefficient
    rf : float
        Risk-free rate
    
    Returns:
    --------
    w : array
        Optimal portfolio weights
    """
    n = Sigma.shape[0]
    w = cp.Variable(n)
    
    # Objective: Minimize lambda * w' * Sigma * w - (1/A) * (mu' * w - rf)
    objective = cp.Minimize(lmd * cp.quad_form(w, Sigma) - (1/A) * (mu.T @ w - rf))
    
    # Constraints: weights >= 0 and sum(weights) = 1
    #constraints = [w >= 0, cp.sum(w) == 1]
    constraints = [ cp.sum(w) == 1]
    
    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return w.value

In [22]:
# Calculate Black-Litterman portfolio weights
w_BL = Markowitz(mu=mu_bar, Sigma=Sigma, A=A, rf=rf)

print("Black-Litterman Portfolio Weights:")
w_BL_series = pd.Series(w_BL, index=countries)
print(w_BL_series.round(4))

print("\nComparison with Initial Weights:")
comparison = pd.DataFrame({
    'Initial Weights': w0,
    'BL Weights': w_BL,
    'Difference': w_BL - w0
}, index=countries)
print(comparison.round(4))

Black-Litterman Portfolio Weights:
Australia    0.147000
Canada       0.050000
France       0.048100
Germany      0.398400
Japan        0.167200
UK          -0.494300
USA          0.683500
dtype: float64

Comparison with Initial Weights:
           Initial Weights  BL Weights  Difference
Australia         0.016000    0.147000    0.131000
Canada            0.022000    0.050000    0.028000
France            0.052000    0.048100   -0.003900
Germany           0.055000    0.398400    0.343400
Japan             0.116000    0.167200    0.051200
UK                0.124000   -0.494300   -0.618300
USA               0.615000    0.683500    0.068500


## Tracking Error Volatility

In [18]:
# Tracking error volatility
# TE_vol = sqrt((x-x0)' * Sigma * (x-x0))
# x = BL weights; x0 = benchmark weights

x = w_BL - w0
TE_vol = np.sqrt(x.T @ Sigma @ x)

print(f"Tracking Error Volatility: {TE_vol*100:.4f}%")

Tracking Error Volatility: 2.8120%
